In [4]:
from covid.common import SparkSessionService

In [5]:
workspace_path = "COVID-DATABRICKS"
database_path = f"/{workspace_path}/.data"
spark_session_service = SparkSessionService()
spark = spark_session_service.get_spark_session()

In [6]:
from covid.bronze import ingest_raw_files

landing_path = f"{database_path}/raw/"
bronze_path = f"{database_path}/bronze/covid"

ingest_raw_files(spark, landing_path, bronze_path)

25/11/16 19:58:35 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [7]:
covid_df = spark.read.format("delta").load(bronze_path)
covid_df.show()

+------+------+---------+-----+------+--------------+---------------+----------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+----------------------+
|regiao|estado|municipio|coduf|codmun|codRegiaoSaude|nomeRegiaoSaude|      data|semanaEpi|populacaoTCU2019|casosAcumulado|casosNovos|obitosAcumulado|obitosNovos|Recuperadosnovos|emAcompanhamentoNovos|interior/metropolitana|
+------+------+---------+-----+------+--------------+---------------+----------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+----------------------+
|Brasil|  null|     null|   76|  null|          null|           null|2025-01-01|        1|       210147125|   3.9073544E7|         0|         714597|          0|        38340436|                18511|                  null|
|Brasil|  null|     null|   76|  null|          null|           null|2025-01-02|        1|       2101471

In [11]:
covid_df.count()

11177059

In [ ]:
import pyspark.sql.functions as F

covid_df.where(F.col("municipio").isNotNull()).count()

11078730